In [3]:
#Setup and Data Loading
import piplite
await piplite.install(['folium', 'pandas'])
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Load SpaceX launch data with geographical coordinates
from js import fetch
import io
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df = pd.read_csv(spacex_csv_file)

# Extract launch site locations
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()[['Launch Site', 'Lat', 'Long']]

In [4]:
#Task 1: Mark All Launch Sites

# Initialize map centered on Florida (where most launch sites are)
site_map = folium.Map(location=[28.5, -81.0], zoom_start=5)

# Add markers for each launch site
for index, site in launch_sites_df.iterrows():
    # Create circle marker
    folium.Circle(
        location=[site['Lat'], site['Long']],
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#d35400'
    ).add_child(folium.Popup(site['Launch Site'])).add_to(site_map)
    
    # Create text label
    folium.map.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

site_map

In [ ]:
#Key Observations:

All launch sites are near the equator (between 28-34°N latitude)

All sites are very close to coastlines (within 5km)

Sites are located in Florida (CCAFS, KSC) and California (VAFB)

In [6]:
#Task 2: Success/Failure Markers

# Create color column based on launch outcome
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize map with marker clusters
site_map = folium.Map(location=[28.5, -81.0], zoom_start=5)
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"
    ).add_to(marker_cluster)

site_map

In [8]:
#Success Rate Analysis:

KSC LC-39A has the highest success rate (most green markers)

VAFB SLC-4E has fewer launches but good success rate

CCAFS SLC-40 shows improvement over time (more green in later flights)

In [10]:
#Task 3: Proximity Analysis

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

# Example for CCAFS SLC-40
launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0]
coastline = [28.56367, -80.57163]  # Coordinates found using MousePosition

# Calculate distance
distance = calculate_distance(launch_site['Lat'], launch_site['Long'], *coastline)

# Add distance marker
distance_marker = folium.Marker(
    coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Add line
folium.PolyLine(
    locations=[[launch_site['Lat'], launch_site['Long']], coastline],
    color='blue',
    weight=2
).add_to(site_map)

site_map

In [ ]:
#Proximity Findings:

Coastline: All sites are within 5km of coastline for safety

Cities: Sites maintain 10-30km distance from major cities

Transportation:

CCAFS and KSC have highways within 5km

VAFB has railway within 10km

Safety Zones: Clear buffer zones around launch complexes

In [12]:
#Complete Proximity Analysis Code

# Define proximity points for each launch site (found using MousePosition)
proximity_points = {
    'CCAFS SLC-40': {
        'coastline': [28.56367, -80.57163],
        'highway': [28.56353, -80.57085],
        'city': [28.612, -80.80788]  # Titusville
    },
    'KSC LC-39A': {
        'coastline': [28.58326, -80.59391],
        'highway': [28.57353, -80.65542],
        'city': [28.612, -80.80788]  # Titusville
    },
    'VAFB SLC-4E': {
        'coastline': [34.63276, -120.62509],
        'railway': [34.63614, -120.62429],
        'city': [34.63813, -120.45856]  # Lompoc
    }
}

# Create map with all proximity markers
site_map = folium.Map(location=[34.63276, -120.62509], zoom_start=5)

# Add launch sites
for index, site in launch_sites_df.iterrows():
    folium.CircleMarker(
        location=[site['Lat'], site['Long']],
        radius=8,
        color='black',
        fill=True,
        fill_color='orange',
        popup=site['Launch Site']
    ).add_to(site_map)

# Add proximity markers and lines
for site_name, points in proximity_points.items():
    site = launch_sites_df[launch_sites_df['Launch Site'] == site_name].iloc[0]
    for point_type, coords in points.items():
        distance = calculate_distance(site['Lat'], site['Long'], *coords)
        
        # Add marker
        folium.Marker(
            coords,
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 10; color:blue;"><b>{point_type.title()}<br>{distance:.2f} KM</b></div>'
            )
        ).add_to(site_map)
        
        # Add line
        folium.PolyLine(
            locations=[[site['Lat'], site['Long']], coords],
            color='blue',
            weight=1,
            dash_array='5,5'
        ).add_to(site_map)

site_map

In [13]:
# Initialize map centered on Florida launch sites
site_map = folium.Map(location=[28.5, -81.0], zoom_start=5)

# Define coordinates for each launch site's proximities (obtained via MousePosition)
proximity_data = {
    'CCAFS SLC-40': {
        'launch_coords': [28.563197, -80.576820],
        'city': {'coords': [28.612, -80.80788], 'name': 'Titusville', 'distance': None},
        'railway': {'coords': [28.57208, -80.58527], 'name': 'Railway', 'distance': None},
        'highway': {'coords': [28.56353, -80.57085], 'name': 'Highway 401', 'distance': None},
        'coastline': {'coords': [28.56367, -80.57163], 'name': 'Atlantic Coast', 'distance': None}
    },
    'KSC LC-39A': {
        'launch_coords': [28.573255, -80.646895],
        'city': {'coords': [28.612, -80.80788], 'name': 'Titusville', 'distance': None},
        'highway': {'coords': [28.57353, -80.65542], 'name': 'Highway 3', 'distance': None},
        'coastline': {'coords': [28.58326, -80.59391], 'name': 'Atlantic Coast', 'distance': None}
    },
    'VAFB SLC-4E': {
        'launch_coords': [34.632834, -120.610745],
        'city': {'coords': [34.63813, -120.45856], 'name': 'Lompoc', 'distance': None},
        'railway': {'coords': [34.63614, -120.62429], 'name': 'Coast Line', 'distance': None},
        'coastline': {'coords': [34.63276, -120.62509], 'name': 'Pacific Coast', 'distance': None}
    }
}

# Calculate distances and add to map
for site, data in proximity_data.items():
    # Add launch site marker
    folium.CircleMarker(
        location=data['launch_coords'],
        radius=8,
        color='black',
        fill=True,
        fill_color='orange',
        popup=f"<b>{site}</b>"
    ).add_to(site_map)
    
    # Process each proximity type
    for prox_type in ['city', 'railway', 'highway', 'coastline']:
        if prox_type in data:
            # Calculate distance
            dist = calculate_distance(
                data['launch_coords'][0], data['launch_coords'][1],
                data[prox_type]['coords'][0], data[prox_type]['coords'][1]
            )
            data[prox_type]['distance'] = dist
            
            # Add proximity marker with custom icon
            icon_html = f"""
            <div style="
                font-size: 10pt; 
                color: {'red' if prox_type == 'city' else 'blue'};
                background: white;
                padding: 2px;
                border-radius: 3px;
                border: 1px solid {'red' if prox_type == 'city' else 'blue'};
            ">
                <b>{data[prox_type]['name']}<br>{dist:.2f} km</b>
            </div>
            """
            
            folium.Marker(
                location=data[prox_type]['coords'],
                icon=DivIcon(
                    icon_size=(100,30),
                    icon_anchor=(50,15),
                    html=icon_html
                )
            ).add_to(site_map)
            
            # Add connecting line
            folium.PolyLine(
                locations=[data['launch_coords'], data[prox_type]['coords']],
                color='red' if prox_type == 'city' else 'blue',
                weight=1.5,
                dash_array='5,5' if prox_type in ['railway', 'highway'] else None
            ).add_to(site_map)

# Add legend
legend_html = '''
<div style="
    position: fixed; 
    bottom: 50px; 
    left: 50px; 
    width: 150px; 
    height: 120px; 
    background: white;
    border:2px solid grey; 
    z-index:9999; 
    padding: 5px;
    font-size:12px;
">
    <b>Proximity Legend</b><br>
    <i class="fa fa-circle" style="color:red"></i> City<br>
    <i class="fa fa-circle" style="color:blue"></i> Railway<br>
    <i class="fa fa-circle" style="color:blue"></i> Highway<br>
    <i class="fa fa-circle" style="color:blue"></i> Coastline<br>
    <i class="fa fa-circle" style="color:black"></i> Launch Site
</div>
'''
site_map.get_root().html.add_child(folium.Element(legend_html))

# Display map
site_map

In [ ]:
#Strategic Location Advantages:

Equatorial Proximity: Takes advantage of Earth's rotation for fuel efficiency

Coastal Access: Allows for safe launch trajectories over water

Infrastructure: Close to transportation networks for logistics

Safety: Maintains safe distances from populated areas while being accessible

In [ ]:
#Key Features of This Solution:
Comprehensive Proximity Analysis:

Calculates distances to cities, railways, highways, and coastlines

Uses the haversine formula for accurate distance calculations

Custom Visual Markers:

Red markers for cities

Blue markers for infrastructure (railway/highway/coastline)

Orange markers for launch sites

Distance labels on all proximity markers

Clear Map Elements:

Dashed lines for transportation routes

Solid lines for geographical features

Interactive popups with location names

Strategic Findings:

All launch sites are within 5km of major transportation routes

Maintain safe distances (10-30km) from populated areas

Immediate coastline access for all sites

VAFB has closest railway access (1.2km)

CCAFS has closest highway access (0.5km)

User-Friendly Enhancements:

Interactive legend explaining color coding

Clean distance labels (2 decimal places)

Zoom-friendly marker sizes